In [ ]:
# default_exp util

# day 10

## part 1

In [ ]:
def read(f):
    xs = sorted(int(x) for x in open(f, 'r'))
    xs = [0,] + xs + [max(xs) + 3,]
    return xs

In [ ]:
xs_test_1 = sorted(int(x) for x in """16
10
15
5
1
11
7
19
6
12
4""".split('\n'))
xs_test_1 = [0,] + xs_test_1 + [max(xs_test_1) + 3]
xs_test_1

[0, 1, 4, 5, 6, 7, 10, 11, 12, 15, 16, 19, 22]

    0 1 [2 3] 4 5 6 7 [8 9] 10 11 12 [13 14] 15 16 [17 18] 19 [20 21] 22

In [ ]:
xs_test_2 = read("./data/10_test.txt")

In [ ]:
def missing(xs):
    missing = []
    for i in range(min(xs), max(xs) + 1):
        if i not in xs:
            missing = missing + [i,]
    return missing

In [ ]:
def threes(xs):
    gaps = missing(xs)
    return sum(
        b - a == 1
        for (a, b) in zip(gaps, gaps[1:])
    )

In [ ]:
def ones(xs):
    return sum(
        i in xs and i - 1 in xs
        for i in range(min(xs), max(xs) + 1)
    )

Or, a bit faster(?):

In [ ]:
def ones(xs):
    return sum(
        xs[i] + 1 == xs[i + 1]
        for i, _ in enumerate(xs[:-1])
    )

In [ ]:
def p1(xs):
    return ones(xs) * threes(xs)

In [ ]:
assert p1(xs_test_1) == 5 * 7

In [ ]:
assert p1(xs_test_2) == 10 * 22

In [ ]:
xs = read("./data/10.txt")

In [ ]:
p1(xs)

2738

## part 2

Part 2 is a combinatorial problem: in how many ways can non-required adapters be removed from the sequence, such that there is never a gap of more than 3?

For instance, example 1 has 1 group of 2 non-essential adapters and 1 group of 1:

    (0) 1 4 [5 6] 7 10 [11] 12 15 16 19 (22)

This means that total number of arrangements is $4 \cdot 2 = 8$, because:
- for a group of 1 (A), you can remokeep A or remove it: 2 combinations
- group of 2 (AB): you can keep both AB, A, B, or neither: 4
- group of 3 (ABC): you can keep adapter A, or B, or C, or AB, or BC, or AC, or AC, or neither: 7

For example 2:

    (0) 1 [2 3] 4 7 [8 9 10] 11 14 17 [18 19] 20 23 [24] 25 28 31 [32 33 34] 35 38 39 42 45 [46 47 48] 49 (52)

In [ ]:
4 * 7 * 4 * 2 * 7 * 7

10976

And for the full problem:

    (0) 1 2 5 [6 7 8] 9 12 13 16 [17 18] 19 22 25 [26 27] 28 31 34 [35 36 37] 38 41 44 [45 46 47] 48 51 52 53 56 59 [60 61] 62 65 [66 67] 68 71 [72 73 74] 75 78 81 84 87 [88 89] 90 93 [94] 95 98 99 102 [103 104 105] 106 109 [110 111 112] 113 116 [117] 118 121 [122 123] 124 127 130 133 [134 135 136] 137 140 [141] 142 145 [146 147 148] 149 152 153 156 159 [160] 161 164 165 168 [169 170] 171 172 175 178 [179 180 181] 182 (185)

In [ ]:
p2 = 7*4*4*7*7*4*4*7*4*2*7*7*2*4*7*2*7*2*4*7
p2

10578455953408

For input this size, counting by hand is still reasonable.

## both parts again

In [ ]:
from operator import mul
from functools import reduce
from itertools import groupby
from collections import defaultdict

In [ ]:
# export
def pairwise(xs):
    return zip(xs, xs[1:])

In [ ]:
# export
def split(xs, at):
    return [list(group) for k, group in groupby(xs, lambda x: x == at) if not k]

Readable part 1:

In [ ]:
def p1(xs):
    return sum(
        (b - a) == 1
        for (a, b) in pairwise(xs)
    ) * sum(
        (b - a) == 3
        for (a, b) in pairwise(xs)
    )

A bit more tacit part 1:

In [ ]:
def p1(xs):
    diffs = defaultdict(int)
    for (a, b) in pairwise(xs):
        diffs[b - a] += 1
    return reduce(mul, diffs.values())

In [ ]:
assert p1(xs_test_1) == 5 * 7

In [ ]:
assert p1(xs_test_2) == 10 * 22

In [ ]:
p1(xs)

2738

What I tried to do by hand can be done by splitting the list of sorted differences and counting lengths of splits.

Then, same mapping as last time, 3 to 7, 2 to 4, 1 to 2 (and 0 to 1).  Maybe it would be more elegant to use a defaultdict, but a string substitution and eval works as well:

In [ ]:
def p2(xs):
    return eval(
        str([len(group) - 1 for group in split([b - a for (a, b) in pairwise(xs)], 3)])
            .replace('[', '')
            .replace(']', '')
            .replace(',', '*')
            .replace('3', '7')
            .replace('2', '4')
            .replace('1', '2')
            .replace('0', '1')
    )

In [ ]:
assert p2(xs_test_1) == 8

In [ ]:
assert p2(xs_test_2) == 19208

In [ ]:
p2(xs)

74049191673856